<a href="https://colab.research.google.com/github/sabrinajmei/met-artwork-app/blob/main/The_Met_Artwork_App.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project Description



Metropolitan Museum of Art App: This app accesses data from [The Metropolitan Museum of Art Collection API](https://metmuseum.github.io/#objects) to generate an **email spotlighting a specific work of art from The Met**, with corresponding information and background on the artwork.


NOTE: The spotlighted artwork is randomized, and there will be no specificity from personal curation.

# Functionality Overview


We ask the user to input their email and name.

We fetch data from The Met Collection API for a randomized work of art. From there, we process this data to collect the following information about the artwork, along with an image of the piece:

*   **Title**
*  **Artist**
*   **Culture**
*   **Classification**
*   **Medium**
*   **Date Created**

All of this information is formatted and sent to the user's email. At the bottom of the email, we will display 2 buttons:
* The first button will link to a Met web page with more information about the art work.
* The second button will link to The Met's general website.

# Information Requirements

### Inputs:

*  User name
*  User email
*  API response

### Outputs:

*   API request
*   Printed artwork information, along with corresponding image
*   Formatted email containing artwork information to user's inbox

# Setup & Configuration

### Package Import

In [ ]:
import requests
import random
from IPython.display import display, Image
from datetime import datetime

### Authorization and API Access

The Met does not require developers to register or obtain an API key to access the API, so there is no authentication process necessary

In [ ]:
all_objects = "https://collectionapi.metmuseum.org/public/collection/v1/objects"
response_test = requests.get(all_objects)
api_test = response_test.json()

We see that there are 485,596 objects, presumably with IDs from 1-485,596.

### Collecting User Info

In [ ]:
def get_user_email():
    return input("Please enter your email address: ")

def get_user_name():
    return input("Please enter your name: ")

### Email Setup (Mailgun)

In [ ]:
MAILGUN_API_KEY = "fe5e9017ec6c8b9aff0ca4962df4cad1-1c7e8847-08792a92"
MAILGUN_SENDER_ADDRESS = "sjm189@georgetown.edu"
MAILGUN_DOMAIN = "sandbox6f1952847945490d935e1788c1a47f13.mailgun.org"

# Usage

In [ ]:
def get_random_object_id():
    return random.randint(1, 485596)


def get_artwork_summary(object_id):
    url = (
        f"https://collectionapi.metmuseum.org/public/collection/v1/objects/{object_id}"
    )
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        if all(data.get(key) for key in ["primaryImage", "title", "artistDisplayName"]):
            summary = {
                "Title": data["title"],
                "Artist": data["artistDisplayName"],
                "Culture": data.get("culture", "N/A"),
                "Classification": data.get("classification", "N/A"),
                "Object URL": data.get("objectURL", "N/A"),
                "Primary Image URL": data["primaryImage"],
                "Medium": data.get("medium", "N/A"),
                "Dimensions": data.get("dimensions", "N/A"),
                "Date Created": data.get("objectDate", "N/A"),
                "Wikidata URL": data.get("objectWikidata_URL", "N/A"),
                "Artist URL": data.get("artistULAN_URL","N/A")
            }
            return summary
    return None


def create_artwork_paragraph(artwork_summary):
    paragraph = f"This image depicts {artwork_summary['Title']}, an artwork by {artwork_summary['Artist']}. "
    paragraph += f"For more information, visit [Met Museum source]({artwork_summary['Object URL']})"

    # Include Wikidata source if available
    if artwork_summary["Wikidata URL"] and artwork_summary["Wikidata URL"] != "N/A":
        paragraph += f" or [Wikidata source]({artwork_summary['Wikidata URL']})."
    else:
        paragraph += "."

    return paragraph


def display_artwork(artwork_summary, image_size=(300, 300)):
    print("Artwork Summary:")
    for key, value in artwork_summary.items():
        print(f"{key}: {value}")

    # Display the primary image with a specified size
    primary_image_url = artwork_summary.get("Primary Image URL", None)
    if primary_image_url:
        print("\nPrimary Image:")
        display(Image(url=primary_image_url, width=image_size[0], height=image_size[1]))
    else:
        print("\nPrimary Image: Image not available.")

# Sending the Email

In [ ]:
#formatting the current date
current_date = datetime.today().date()
formatted_date = current_date.strftime("%A, %B %d, %Y")

#sending the email
def send_email_with_artwork_and_buttons(
    artwork_summary,
    recipient_address="sjm189@georgetown.edu",
    subject="Snapshot of The Met: Check out this amazing artwork!",
):
    print("SENDING EMAIL TO:", recipient_address)
    print("SUBJECT:", subject)

    # Create HTML content for the email
    html_content = f"<p style='font-size: 24px;'><strong>{formatted_date}</strong>: <strong style='font-size: 24px;'>Snapshot of The Met</strong></p>"
    html_content += f"<p>Hello {get_user_name()}! </p>"
    html_content += f"<p>Check out this amazing artwork:</p>"
    html_content += f"<p>Title: {artwork_summary['Title']}</p>"
    html_content += f"<p>Artist: {artwork_summary['Artist']}</p>"
    html_content += f"<p>Culture: {artwork_summary['Culture']}</p>"
    html_content += f"<p>Classification: {artwork_summary['Classification']}</p>"
    html_content += f"<p>Medium: {artwork_summary['Medium']}</p>"
    html_content += f"<p>Date Created: {artwork_summary['Date Created']}</p>"

    # Include image in the email if available
    if artwork_summary["Primary Image URL"]:
        html_content += f"<p>Image:</p>"
        html_content += f'<img src="{artwork_summary["Primary Image URL"]}" alt="Artwork Image" style="max-width: 500px;">'

    # Include Met Museum link for ART button if available
    if artwork_summary["Object URL"] and artwork_summary["Object URL"] != "N/A":
        html_content += f'<div><a href="{artwork_summary["Object URL"]}" style="background-color: #eb0029; color: white; padding: 10px 15px; margin-bottom: 10px; text-align: center; text-decoration: none; display: inline-block; border-radius: 5px;">Learn more about the artwork here.</a></div>'

    # Include ULAN link for ARTIST button if available
    if artwork_summary["Artist URL"] and artwork_summary["Artist URL"] != "N/A":
        html_content += f'<div><a href="{artwork_summary["Artist URL"]}" style="background-color: #eb0029; color: white; padding: 10px 15px; margin-top: 10px; text-align: center; text-decoration: none; display: inline-block; border-radius: 5px;">Learn more about the artist here.</a></div>'



    try:
        request_url = f"https://api.mailgun.net/v3/{MAILGUN_DOMAIN}/messages"
        message_data = {
            "from": MAILGUN_SENDER_ADDRESS,
            "to": recipient_address,
            "subject": subject,
            "html": html_content,
        }
        response = requests.post(
            request_url, auth=("api", MAILGUN_API_KEY), data=message_data
        )
        print("RESULT:", response.status_code)
        response.raise_for_status()
        print("Email sent successfully!")
        return response.status_code
    except requests.exceptions.RequestException as e:
        print(f"Error sending email: {str(e)}")
        return None


In [ ]:
while True:
    random_object_id = get_random_object_id()
    artwork_summary = get_artwork_summary(random_object_id)

    if artwork_summary:
        display_artwork(artwork_summary)
        paragraph = create_artwork_paragraph(artwork_summary)
        print("\nArtwork Description:")
        print(paragraph)

        # Prompt user for email address
        recipient_email = get_user_email()

        # Send email with artwork summary and buttons
        send_email_with_artwork_and_buttons(artwork_summary, recipient_address=recipient_email)
        break
    else:
        print(
            f"Object ID {random_object_id} does not have complete information or an available image. Trying another one..."
        )

Object ID 320022 does not have complete information or an available image. Trying another one...
Object ID 86526 does not have complete information or an available image. Trying another one...
Artwork Summary:
Title: Spoon
Artist: B. T.
Culture: American
Classification: 
Object URL: https://www.metmuseum.org/art/collection/search/7819
Primary Image URL: https://images.metmuseum.org/CRDImages/ad/original/97996.jpg
Medium: Silver
Dimensions: L. 4 in. (10.2 cm)
Date Created: 1740–50
Wikidata URL: https://www.wikidata.org/wiki/Q116350626
Artist URL: 

Primary Image:



Artwork Description:
This image depicts Spoon, an artwork by B. T.. For more information, visit [Met Museum source](https://www.metmuseum.org/art/collection/search/7819) or [Wikidata source](https://www.wikidata.org/wiki/Q116350626).
Please enter your email address: awaters4730@gmail.com
SENDING EMAIL TO: awaters4730@gmail.com
SUBJECT: Snapshot of The Met: Check out this amazing artwork!
Please enter your name: asdfn
RESULT: 200
Email sent successfully!
